# データとライブラリの読み込み

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold

import pprint

# CatBoostを利用した分類
import lightgbm as lgb

from sklearn.model_selection import GridSearchCV
# 指標を計算するため
from sklearn.metrics import accuracy_score, cohen_kappa_score, make_scorer, f1_score, recall_score


#google drive読み込み
# from google.colab import drive, files
# drive.mount('/content/drive')
# dataPath = "/content/drive/My Drive/01 Lab/novel_bookmark/datasets/"
# test = pd.read_csv(dataPath + 'test.csv')
# train = pd.read_csv(dataPath +'train.csv')
# sub = pd.read_csv(dataPath +'sample_submission.csv')

#local 読み込み
dataPath = './datasets/'
titlePath = './data/titledata/'
keyPath = './data/keyworddata/'
storyPath = './data/storydata/'
allPath = './data/all/'

test = pd.read_csv(dataPath + 'test.csv')
train = pd.read_csv(dataPath +'train.csv')
sub = pd.read_csv(dataPath +'sample_submission.csv')

print(len(train))

40000


# 特徴量エンジニアリング

In [38]:
test = pd.read_csv(dataPath + 'test.csv')
train = pd.read_csv(dataPath +'train.csv')
sub = pd.read_csv(dataPath +'sample_submission.csv')

data = pd.concat([train, test], sort=False)#データの統合
data = data.reset_index()#indexが重複するためindexを振り直し

print("len(data):", len(data))

#必要のないカラムを削除
#ncode:小説コード, writer, userid, pc_or_k:小説の投稿機種
delete_columns = ['ncode', 'writer', 'userid', 'pc_or_k', 'general_firstup']
data.drop(delete_columns, axis=1, inplace=True)

#novel_type (1, 2)を(0, 1)に変更
data['novel_type'] = data['novel_type'].map({1: 0, 2: 1})

#'general_firstup'(初回投稿日)を今日までの日数に変換
import datetime
def calc_diff_days(x):#return days(int)
  y_m_d = x.split(" ")[0]
  y, m, d = map(int, y_m_d.split("-"))
  ymd = datetime.datetime(y, m, d)
  diff = datetime.datetime.today() - ymd
  return diff.days
# data['general_firstup'] = data['general_firstup'].map(calc_diff_days)

#テキストデータを削除
text_columns = ['title','story', 'keyword']
data.drop(text_columns, axis=1, inplace=True)

#titleデータをベクトルに変換したものを追加
# title_data = pd.read_csv(titlePath+'title_vec_learnedModel.csv', index_col=0)#学習済みモデル
# title_data = pd.read_csv(titlePath+'title_vec_originalLearned_300.csv', index_col=0)#自作辞書モデル
title_data = pd.read_csv(allPath+'title_mecab_vec_tdidf_50.csv', index_col=0)#自作辞書モデル
data = pd.merge(data, title_data, left_index=True, right_index=True, sort=False, how='inner')

#keywordデータをベクトルに変換したものを追加
# key_data = pd.read_csv(keyPath+'key_vec_learnedModel.csv', index_col=0)#学習済みモデル
# key_data = pd.read_csv(keyPath+'key_vec_originalModel.csv', index_col=0)#自作辞書モデル
key_data = pd.read_csv(allPath+'key_vec_mecab_tdidf_50.csv', index_col=0)#自作辞書モデル
data = pd.merge(data, key_data, left_index=True, right_index=True, sort=False, how='inner')

#storyデータをベクトルに変換したものを追加
# story_data = pd.read_csv(storyPath+'vec_learnedModel.csv', index_col=0)#学習済みモデル
story_data = pd.read_csv(allPath+'story_vec_mecab_tdidf_50.csv', index_col=0)#自作辞書モデル
data = pd.merge(data, story_data, left_index=True, right_index=True, sort=False, how='inner')

# storyデータの文字数を追加
# count_story_data = pd.read_csv(storyPath+ 'count.csv')
# data = pd.merge(data, count_story_data, left_index=True, right_index=True, sort=False, how='inner')

import re
data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

data = data.fillna(0)

data.drop('index', axis=1, inplace=True)

#dataをtest, trainに分割
train = data[:len(train)]
test = data[len(train):]


len(data): 48522


In [20]:
train.columns
train.index

Int64Index([    7,     9,    10,    11,    14,    15,    21,    58,    64,
               70,
            ...
            39253, 39299, 39464, 39475, 39659, 39671, 39786, 39787, 39793,
            39979],
           dtype='int64', length=4070)

# サンプリング

## アンダーサンプリング

In [19]:
#一時的に欠損値を含むものを削除
# train = train.dropna(how='any')

train0 = train[train['fav_novel_cnt_bin']==0]
train1 = train[train['fav_novel_cnt_bin']==1]
train2 = train[train['fav_novel_cnt_bin']==2]
train3 = train[train['fav_novel_cnt_bin']==3]
train4 = train[train['fav_novel_cnt_bin']==4]


#todo train list 等に突っ込んでアンダーサンプリングでloggingする
num = len(train4)
train = pd.concat([train0[:num], train1[:num], train2[:num], train3[:num], train4[:num]], sort=False)#データの統合
print(len(train))

4070


## オーバーサンプリング

### SMOTE

In [39]:
#data分割
train = train.fillna(0)

x_train = train.drop('fav_novel_cnt_bin', axis=1)#訓練データ
y_train = train['fav_novel_cnt_bin']#訓練データの答え

# from sklearn.model_selection import train_test_split
# x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.3, random_state=0, stratify=y_train)

from imblearn.over_sampling import SMOTE
sm = SMOTE()
x_resampled, y_resampled = sm.fit_resample(x_train, y_train)
x_train = x_resampled
y_train = y_resampled

### 復元抽出

In [6]:
train0 = train[train['fav_novel_cnt_bin']==0]
train1 = train[train['fav_novel_cnt_bin']==1]
train2 = train[train['fav_novel_cnt_bin']==2]
train3 = train[train['fav_novel_cnt_bin']==3]
train4 = train[train['fav_novel_cnt_bin']==4]

num_train0 = len(train0)
train = pd.concat(
  [
    train0.sample(n=num_train0, replace=True),
    train1.sample(n=num_train0, replace=True),
    train2.sample(n=num_train0, replace=True),
    train3.sample(n=num_train0, replace=True),
    train4.sample(n=num_train0, replace=True),
  ],
  sort=False
)

# 機械学習


### 訓練データ

In [4]:
#data分割
x_train = train.drop('fav_novel_cnt_bin', axis=1)#訓練データ
y_train = train['fav_novel_cnt_bin']#訓練データの答え

### 訓練データと答えに分割

In [7]:
#data分割
x_train = train.drop('fav_novel_cnt_bin', axis=1)#訓練データ
y_train = train['fav_novel_cnt_bin']#訓練データの答え

from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.3, random_state=0, stratify=y_train)

## 学習モデル

### 勾配ブースティング

In [40]:
import lightgbm as lgb

# モデルの学習
model = lgb.LGBMClassifier(
  learning_rate=0.05,
  num_leaves=300,
  max_depth=75
)
model.fit(x_train, y_train)

# テストデータの予測クラス (予測クラス(0 or 1 or 2)を返す)
# y_pred = model.predict(x_valid)

# テストデータのクラス予測確率 (各クラスの予測確率 [クラス0の予測確率,クラス1の予測確率,クラス2の予測確率] を返す)
# y_pred_prob = model.predict_proba(x_valid)

LGBMClassifier(learning_rate=0.05, max_depth=75, num_leaves=300)

### ランダムフォレスト

In [37]:
from sklearn.ensemble import RandomForestClassifier

randomforest = RandomForestClassifier()
randomforest.fit(x_train, y_train)
# y_pred = randomforest.predict(x_valid)
# y_pred_prob = randomforest.predict_proba(x_valid)

RandomForestClassifier()

### ロジスティック回帰

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(x_train, y_train)
# pred = logreg.predict(x_valid)
# y_pred = logreg.predict(x_valid)
# y_pred_prob = logreg.predict_proba(x_valid)

In [ ]:
#正答率の確認
from sklearn.metrics import accuracy_score
def calc_score(y_true, y_pred):
  tmp = []
  for i in range(len(y_true)):
    tmp.append([0,0,0,0,0])
    tmp[i][int(y_true[i])] = 1
  tmp = np.array(tmp)
  #確率が極端な場合クリップ（損失が大きくなりすぎるため）
  y_pred = np.where(y_pred<=0.001, 0.001, y_pred)
  y_pred = np.where(y_pred>=0.999, 0.999, y_pred)
  return -np.sum(tmp * np.log(y_pred)) / len(y_true)

print('accuracy = ', accuracy_score(y_true=y_valid.values, y_pred=y_pred))
print('score = ', calc_score(y_true=y_valid.values, y_pred=y_pred_prob))

## submission用に変換

In [37]:

# model = randomforest

test = test.drop('fav_novel_cnt_bin', axis=1)
test = test.fillna(0)
y_pred = model.predict(test)
y_pred_prob = model.predict_proba(test)

#確率が極端な場合クリップ（損失が大きくなりすぎるため）
y_pred_prob = np.where(y_pred_prob<=0.001, 0.001, y_pred_prob)
y_pred_prob = np.where(y_pred_prob>=0.999, 0.999, y_pred_prob)

predData = pd.DataFrame(y_pred_prob)
sub = pd.read_csv(dataPath +'sample_submission.csv')
sub['proba_0'] = predData[0]
sub['proba_1'] = predData[1]
sub['proba_2'] = predData[2]
sub['proba_3'] = predData[3]
sub['proba_4'] = predData[4]

import datetime
now = datetime.datetime.now()
filename = str(now.year)+'-'+str(now.month)+'-'+ str(now.day) +'-'+ str(now.hour)+'-'+str(now.minute)
sub.to_csv('./data/sub/'+ filename +'.csv', index=False)


In [31]:
# 特徴量重要度の算出 (データフレームで取得)
# データフレームを綺麗に出力する関数
import IPython
def display(*dfs, head=True):
    for df in dfs:
        IPython.display.display(df.head() if head else df)

cols = list(train.drop(['fav_novel_cnt_bin'],axis=1).columns)         # 特徴量名のリスト(目的変数CRIM以外)
# f_importance = np.array(model.feature_importance()) # 特徴量重要度の算出
# f_importance = f_importance / np.sum(f_importance)  # 正規化(必要ない場合はコメントアウト)
# df_importance = pd.DataFrame({'feature':cols, 'importance':f_importance})
# df_importance = df_importance.sort_values('importance', ascending=False) # 降順ソート

importance = pd.DataFrame(model.feature_importances_, index=cols, columns=['importance'])
# display(importance)
importance.to_csv('./tmp/importance.csv')


In [12]:
cols

['index',
 'biggenre',
 'genre',
 'novel_type',
 'end',
 'isstop',
 'isr15',
 'isbl',
 'isgl',
 'iszankoku',
 'istensei',
 'istenni',
 '0_x',
 '1_x',
 '2_x',
 '3_x',
 '4_x',
 '5_x',
 '6_x',
 '7_x',
 '8_x',
 '9_x',
 '10_x',
 '11_x',
 '12_x',
 '13_x',
 '14_x',
 '15_x',
 '16_x',
 '17_x',
 '18_x',
 '19_x',
 '20_x',
 '21_x',
 '22_x',
 '23_x',
 '24_x',
 '25_x',
 '26_x',
 '27_x',
 '28_x',
 '29_x',
 '30_x',
 '31_x',
 '32_x',
 '33_x',
 '34_x',
 '35_x',
 '36_x',
 '37_x',
 '38_x',
 '39_x',
 '40_x',
 '41_x',
 '42_x',
 '43_x',
 '44_x',
 '45_x',
 '46_x',
 '47_x',
 '48_x',
 '49_x',
 '0_y',
 '1_y',
 '2_y',
 '3_y',
 '4_y',
 '5_y',
 '6_y',
 '7_y',
 '8_y',
 '9_y',
 '10_y',
 '11_y',
 '12_y',
 '13_y',
 '14_y',
 '15_y',
 '16_y',
 '17_y',
 '18_y',
 '19_y',
 '20_y',
 '21_y',
 '22_y',
 '23_y',
 '24_y',
 '25_y',
 '26_y',
 '27_y',
 '28_y',
 '29_y',
 '30_y',
 '31_y',
 '32_y',
 '33_y',
 '34_y',
 '35_y',
 '36_y',
 '37_y',
 '38_y',
 '39_y',
 '40_y',
 '41_y',
 '42_y',
 '43_y',
 '44_y',
 '45_y',
 '46_y',
 '47_y',
 '48